# Maxwell's Daemon
### Final Project for Computer Based Physical Modelling
### Thomas de Paula Barbosa - 3749592

# Theory
## About the Daemon
Maxwell's daemon is a thought experiment made to test the validity the Second Law of Thermodynamics. The thought experiment consists in two isolated boxes filled with gas which have a gate in between them. The gate's opening and closing is controlled by a daemon who only allows molecules that have a certain velocity to pass through the gate. After a set amount of time, this means that faster (hotter) molecules will be on one side, while the slower (colder) molecules will be in the other side. Assuming this could actually happen, one of the system's container would decrease in temperature and the other one would increase. Since we would be ordering an unordered system without adding energy, this would actually decrease its entropy, which is a direct violation of the Second Law of Thermodynamics.

## Brief Note on Code
While Python is dynamically typed, I like giving type hints for keeping track of things more easily. The objective of this code is to simulate Maxwell's Daemon and observe what happens to the temperature in each container.

## Collisions
In the simulation, all collisions are perfectly elastic and momentum is conserved. Since all the masses are the same, the resulting velocity of each particle is as follows:
$$
P_i = P_f\\
m_1 u_1 + m_2 u_2 = m_1 v_1 + m_2 v_2\\
u_1 + u_2 = v_1 + v_2
$$

As for kinetic energy, we have

$$
K_i = K_f\\
\frac{1}{2}m_1 u_1^2 + \frac{1}{2}m_2 u_2^2 = \frac{1}{2}m_1 v_1^2 + \frac{1}{2}m_2 v_2^2
$$

Solving for $v_1$ and $v_2$ we have

$$
v_1 = \frac{m_1 - m_2}{m_1 + m_2}u_1 + \frac{2m_2}{m_1 + m_2}u_2\\
v_2 = \frac{m_2 - m_1}{m_1 + m_2}u_2 + \frac{2m_1}{m_1 + m_2}u_1
$$

Since both masses are the same, the solution becomes:

$$
v_1 = u_2 \\
v_2 = u_1
$$
In code, this means that we can (and will) simply exchange the velocity of the first particle with the velocity of the second particle.

## Creating the Canvas

In [1]:
from ipycanvas import hold_canvas, Canvas, MultiCanvas
import numpy as np


CONTAINER_COORDINATES = np.array([800, 600])  # x, y
# the daemon will open the gate for a particle when it reaches this speed
SPEED_THRESHHOLD: float = 10.0
GATE_SIZE: int = 100
X, Y = 0, 1  # this is just for ease of reading
PARTICLE_RADIUS: int = 5

In [2]:
%load_ext autoreload
%autoreload complete
from particle_class import *
molecules = []
N_PARTICLES = 50
for _ in range(N_PARTICLES):
    molecules.append(
        Particle(
            position=np.array([
                # TODO ensure they don't generate on the limits/borders
                np.random.uniform(2*PARTICLE_RADIUS, CONTAINER_COORDINATES[X] - 2*PARTICLE_RADIUS + 1),
                np.random.uniform(2*PARTICLE_RADIUS, CONTAINER_COORDINATES[Y] - 2*PARTICLE_RADIUS + 1),
            ]),
            velocity=np.random.random(2) * np.random.choice([-1, 1], size=2) * 8,
            radius=PARTICLE_RADIUS,
            #angle=np.random.uniform(0, 2*np.pi)
        )
    )

In [3]:
# NOTE ipycanvas seems to have a bug where numbers must be converted to python's int
# this happens despite the fact that the documentation claims it should be compatible with numpy
cc = Canvas(width=CONTAINER_COORDINATES[X], height=CONTAINER_COORDINATES[Y])
container_middle = int(CONTAINER_COORDINATES[X]/2), int(CONTAINER_COORDINATES[Y]/2)
STEPS = 2_000
with hold_canvas(cc):
    for _ in range(STEPS):
        cc.clear()
        
        # box
        cc.stroke_style = "black"
        cc.stroke_rect(1, 1, int(CONTAINER_COORDINATES[X]-1), int(CONTAINER_COORDINATES[Y]-1))
        # middle line
        cc.stroke_line(
            container_middle[X], 0, 
            container_middle[X], int(container_middle[Y] - GATE_SIZE/2)
        )
        cc.stroke_line(
            container_middle[X], int(CONTAINER_COORDINATES[Y]),
            container_middle[X], int(container_middle[Y] + GATE_SIZE/2)
        )
        # gate/daemon
        cc.stroke_style = "#9c06ec"  # purple
        cc.stroke_line(
            container_middle[X], int(container_middle[Y] - GATE_SIZE/2),
            container_middle[X], int(container_middle[Y] + GATE_SIZE/2)
        )
        
        for i, m in enumerate(molecules):
            m.update()
            for p in molecules:
                m.collide(p)
            if m.isHot:
                cc.fill_style = "red"
            else:
                cc.fill_style = "blue"
            # draw molecule
            cc.fill_circle(int(m.position[X]), int(m.position[Y]), PARTICLE_RADIUS)
        cc.sleep(20)
display(cc)

Canvas(height=600, width=800)

In [4]:
good_side = 0
escaped = 0
for m in molecules:
    if m._isOnCorrectSide():
        good_side += 1
    if m.position[X] > CONTAINER_COORDINATES[X] or m.position[Y] > CONTAINER_COORDINATES[Y]:
        escaped += 1
        #raise RuntimeError("A molecule breached containment!")
    elif m.position[X] < 0 or m.position[Y] < 0:
        escaped += 1
        #raise RuntimeError("A molecule breached containment!")
incorrect_molecules = len(molecules) - good_side
percentage = escaped/len(molecules) * 100
print(f"Good side: {good_side}\nBad side: {incorrect_molecules}")
print(f"{escaped} molecule(s) escaped. That's {percentage:.2f}%")
if escaped >= 5:
    raise RuntimeError("Too many escaped")
elif escaped == 0:
    print("Success!!")

Good side: 47
Bad side: 3
3 molecule(s) escaped. That's 6.00%
